In [1]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v4.min'
}});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [11]:
import msprime
import random
from visualizer import visualizer


# Generate a random tree sequence with record_full_arg=True so that you get marked recombination nodes
rs = random.randint(0,10000)   
ts = msprime.sim_ancestry(
    samples=2,
    recombination_rate=1e-8,
    sequence_length=3_000,
    population_size=10_000,
    record_full_arg=True,
    random_seed=rs
)

#print(ts.tables)

print("random seed:", rs)
print(ts.draw_text())
d3arg = visualizer.D3ARG(ts=ts)
d3arg.draw(width=500, height=500, y_axis_scale="time", y_axis_labels=True)

random seed: 1254
28769.76┊     6   ┊  
        ┊   ┏━┻━┓ ┊  
4607.54 ┊   5   ┃ ┊  
        ┊ ┏━┻┓  ┃ ┊  
3451.36 ┊ ┃  4  ┃ ┊  
        ┊ ┃ ┏┻┓ ┃ ┊  
0.00    ┊ 0 1 3 2 ┊  
        0       3000 

